In [1]:
import pandas as pd

In [2]:
# Importo los csv con los datos
df_encuesta = pd.read_csv('tp-2020-2c-train-cols2.csv')
df_prediccion = pd.read_csv('tp-2020-2c-train-cols1.csv')

In [3]:
# Inspeccionamos las columnas que forman los datos
df_encuesta.head()

,tipo_de_sala,nombre,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede
0,4d,Señor Camilo Pedro,117,hombre,73.5,0,0,59258;,1,NaN,fiumark_quilmes
1,4d,Señora Raquel Angelica,658,mujer,35.0,1,1,586:6;,2,NaN,fiumark_quilmes
2,normal,Señor Antonio Federico,794,hombre,NaN,0,0,"RE""39822",3,NaN,fiumark_chacarita
3,4d,Señor Osvaldo Aureliano,455,hombre,NaN,0,0,"C17""4:39",1,NaN,fiumark_palermo
4,4d,Señorita Rita Eudosia,173,mujer,4.0,1,1,569964,2,NaN,fiumark_palermo


In [4]:
# Inspeccionamos las dimensiones iniciales del dataframe
df_encuesta.shape

(801, 11)

In [5]:
# Eliminamos las columna "id_ticket" y "nombre" porque no nos aportan informacion relavante para la prediccion
df_encuesta.drop(columns='id_ticket', inplace=True)
df_encuesta.drop(columns='nombre', inplace=True)

In [6]:
# DEBUG
df_encuesta.head()

,tipo_de_sala,id_usuario,genero,edad,amigos,parientes,precio_ticket,fila,nombre_sede
0,4d,117,hombre,73.5,0,0,1,NaN,fiumark_quilmes
1,4d,658,mujer,35.0,1,1,2,NaN,fiumark_quilmes
2,normal,794,hombre,NaN,0,0,3,NaN,fiumark_chacarita
3,4d,455,hombre,NaN,0,0,1,NaN,fiumark_palermo
4,4d,173,mujer,4.0,1,1,2,NaN,fiumark_palermo


In [7]:
# Analizamos la integridad de las columnas amigos y parientes 
# Verificamos que no tengan valores nulos y que solo tengan valores numericos enteros positivos
# Para cear una nueva columna 'cantidad_acompanantes' que sea la suma de estas dos

In [8]:
df_encuesta['amigos'].isnull().sum()

0

In [9]:
df_encuesta['parientes'].isnull().sum()

0

In [10]:
df_encuesta['amigos'].value_counts()

0    557
1    179
2     25
4     15
3     14
8      6
5      5
Name: amigos, dtype: int64

In [11]:
df_encuesta['parientes'].value_counts()

0    607
1    110
2     70
3      5
5      4
4      4
6      1
Name: parientes, dtype: int64

In [12]:
df_encuesta['cantidad_acompanantes'] = df_encuesta['amigos'] + df_encuesta['parientes']

In [13]:
# DEBUG
df_encuesta.head()

,tipo_de_sala,id_usuario,genero,edad,amigos,parientes,precio_ticket,fila,nombre_sede,cantidad_acompanantes
0,4d,117,hombre,73.5,0,0,1,NaN,fiumark_quilmes,0
1,4d,658,mujer,35.0,1,1,2,NaN,fiumark_quilmes,2
2,normal,794,hombre,NaN,0,0,3,NaN,fiumark_chacarita,0
3,4d,455,hombre,NaN,0,0,1,NaN,fiumark_palermo,0
4,4d,173,mujer,4.0,1,1,2,NaN,fiumark_palermo,2


In [14]:
# Elimino las columnas amigos y parientes
df_encuesta.drop(columns='amigos', inplace=True)
df_encuesta.drop(columns='parientes', inplace=True)

In [15]:
# DEBUG
df_encuesta.head()

,tipo_de_sala,id_usuario,genero,edad,precio_ticket,fila,nombre_sede,cantidad_acompanantes
0,4d,117,hombre,73.5,1,NaN,fiumark_quilmes,0
1,4d,658,mujer,35.0,2,NaN,fiumark_quilmes,2
2,normal,794,hombre,NaN,3,NaN,fiumark_chacarita,0
3,4d,455,hombre,NaN,1,NaN,fiumark_palermo,0
4,4d,173,mujer,4.0,2,NaN,fiumark_palermo,2


In [16]:
# Analizamos la columna 'tipo_de_sala'
# Verificamos que no tenga valores nulos
df_encuesta['tipo_de_sala'].isnull().sum()

0

In [17]:
df_encuesta['tipo_de_sala'].value_counts()

4d        447
normal    187
3d        167
Name: tipo_de_sala, dtype: int64

In [18]:
# Reemplazamos normal por 2d
df_encuesta['tipo_de_sala'].replace(to_replace='normal', value='2d', inplace=True)

In [19]:
# DEBUG
df_encuesta['tipo_de_sala'].value_counts()

4d    447
2d    187
3d    167
Name: tipo_de_sala, dtype: int64

In [20]:
# DEBUG
df_encuesta.head()

,tipo_de_sala,id_usuario,genero,edad,precio_ticket,fila,nombre_sede,cantidad_acompanantes
0,4d,117,hombre,73.5,1,NaN,fiumark_quilmes,0
1,4d,658,mujer,35.0,2,NaN,fiumark_quilmes,2
2,2d,794,hombre,NaN,3,NaN,fiumark_chacarita,0
3,4d,455,hombre,NaN,1,NaN,fiumark_palermo,0
4,4d,173,mujer,4.0,2,NaN,fiumark_palermo,2


In [21]:
# Analizamos la columna 'id_usuario' es busca de usuarios repetidos
df_encuesta['id_usuario'].duplicated(keep=False).value_counts()

False    801
Name: id_usuario, dtype: int64

In [22]:
# Verifico que no tenga valores nulos
df_encuesta['id_usuario'].isnull().sum()

0

In [23]:
# Analizamos la columna 'nombre_sede', verificamos que no tenga valores nulos
df_encuesta['nombre_sede'].isnull().sum()

2

In [24]:
# En este caso como solo tiene 2 filas con valores null las descartamos
bool_nombre_sede = pd.notnull(df_encuesta['nombre_sede'])  
df_encuesta = df_encuesta[bool_nombre_sede]  

In [25]:
df_encuesta['nombre_sede'].value_counts()

fiumark_palermo      579
fiumark_chacarita    149
fiumark_quilmes       71
Name: nombre_sede, dtype: int64

In [26]:
# DEBUG
df_encuesta.shape

(799, 8)

In [27]:
# Verifico que la columna genero no tenga nulos y solo tenga valores mujer/hombre
df_encuesta['genero'].isnull().sum()

0

In [28]:
df_encuesta['genero'].value_counts()

hombre    513
mujer     286
Name: genero, dtype: int64

In [29]:
# Analizamos la columna 'fila'
df_encuesta['fila'].isnull().sum()

624

In [30]:
# Como la mayoria de los valores de esta columna son nulos decidimos eliminarla
df_encuesta.drop(columns='fila', inplace=True)

In [31]:
# DEBUG
df_encuesta.head()

,tipo_de_sala,id_usuario,genero,edad,precio_ticket,nombre_sede,cantidad_acompanantes
0,4d,117,hombre,73.5,1,fiumark_quilmes,0
1,4d,658,mujer,35.0,2,fiumark_quilmes,2
2,2d,794,hombre,NaN,3,fiumark_chacarita,0
3,4d,455,hombre,NaN,1,fiumark_palermo,0
4,4d,173,mujer,4.0,2,fiumark_palermo,2


In [32]:
# Analizamos la columna 'edad' 
# Tiene nulos pero no tantos teniendo en cuenta el tamano del dataset
# Falta analizar la columna precio con lo de la clase practica del 07/10
df_encuesta['edad'].isnull().sum()

160

In [33]:
# Analizamos el dataframe df_prediccion, verificando que no tenga valores null
df_prediccion.head()

,id_usuario,volveria
0,117,0
1,658,0
2,794,0
3,455,0
4,173,1


In [34]:
df_prediccion.shape

(801, 2)

In [35]:
df_prediccion.isnull().sum()

id_usuario    0
volveria      0
dtype: int64

In [36]:
# Analizamos la columna 'id_usuario' es busca de usuarios repetidos
df_prediccion['id_usuario'].duplicated(keep=False).value_counts()

False    801
Name: id_usuario, dtype: int64

In [39]:
# Mergeamos los dataframe
df = pd.merge(left = df_encuesta, right = df_prediccion, how = 'inner', on = 'id_usuario')

In [40]:
df.head()

,tipo_de_sala,id_usuario,genero,edad,precio_ticket,nombre_sede,cantidad_acompanantes,volveria
0,4d,117,hombre,73.5,1,fiumark_quilmes,0,0
1,4d,658,mujer,35.0,2,fiumark_quilmes,2,0
2,2d,794,hombre,NaN,3,fiumark_chacarita,0,0
3,4d,455,hombre,NaN,1,fiumark_palermo,0,0
4,4d,173,mujer,4.0,2,fiumark_palermo,2,1


In [41]:
# DEBUG
df.shape

(799, 8)